# Planning observations with `astroplan` - Constraints

In [0]:
import numpy as np
import pandas as pd

import pytz

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation
from astroplan import Observer, FixedTarget

import warnings
warnings.filterwarnings('ignore', category=Warning)

---

# List Comprehensions

List comprehensions provide a concise way to create lists (arrays). Common applications are to make new lists where each element is the result of some operations applied to each member of another sequence.

### For example: Create the list: `[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]`

In [0]:
squares = []                    # create a blank list

for x in range(10):             # foor loop 0 -> 9
    squares.append(x**2)        # calculate x**2 for each x, add to end of list

squares

### You can do the same thing with:

In [0]:
squares = [x**2 for x in range(10)]

squares

### You can include `if` statements:

In [0]:
even_squares = []

for x in range(10):
    
    if (x % 2 == 0):
        even_squares.append(x**2)

even_squares

### You can do the same thing with:

In [0]:
even_squares = [x**2 for x in range(10) if (x % 2 == 0)]

even_squares

---

# Now to observations

## Let us start with a external list of target objects:

In [0]:
target_table = pd.read_csv('./Data/ObjectList.csv')

In [0]:
target_table[0:3]

#### Shortcut to pull all values from a `pandas` table:

In [0]:
table_values = target_table.as_matrix()

In [0]:
table_values[0:3]

In [0]:
targets = [FixedTarget(coord=SkyCoord(ra = RA*u.hourangle, dec = DEC*u.deg), name=Name)
           for Name, RA, DEC in table_values]

In [0]:
targets[0:3]

## Observing Night

### You are the most junior member of the team, so you get stuck observing on Thanksgiving

#### Observing Window:
 * Start: `Nov 22, 2018 12:00:00 UTC`
 * End: `Nov 23, 2018 12:00:00 UTC`

In [0]:
window_start = Time("2018-11-22 12:00:00")
window_end = Time("2018-11-23 12:00:00")

### But, you get a trip to the [Canary Islands](https://en.wikipedia.org/wiki/Roque_de_los_Muchachos_Observatory)

In [0]:
#Gran Telescopio Canarias
#Observatorio Roque de los Muchachos

my_location = Observer(longitude = -17.892059 * u.deg,
                       latitude = 28.756581 * u.deg,
                       elevation = 2396 * u.m,
                       timezone = 'UTC',
                       name = "Gran Telescopio Canarias"
                      )

### What time does the window open locally?

In [0]:
print(window_start.to_datetime(my_location.timezone))

#### It is 12 pm local, so my observations start at the `nearest` astromonical twilight.

In [0]:
observe_start = my_location.twilight_evening_astronomical(window_start, which='nearest')
observe_end = my_location.twilight_morning_astronomical(window_start, which='next')

In [0]:
observing_length = (observe_end - observe_start)

print("You can observe for {0:.1f} tonight".format(observing_length.to(u.h)))

## Plot the objects - Over the observing window

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt

from astroplan import time_grid_from_range
from astroplan.plots import plot_sky, plot_airmass

In [0]:
window_range = [window_start, window_end]
window_time_grid = time_grid_from_range(window_range)

In [0]:
fig,ax = plt.subplots(1,1)
fig.set_size_inches(10,5)

fig.tight_layout()

for my_object in targets:
    ax = plot_airmass(my_object, my_location, window_time_grid)

ax.vlines(observe_start.datetime, 1,3, color='r', linewidth=5)
ax.vlines(observe_end.datetime, 1,3, color='r', linewidth=5)

ax.legend(loc=0,shadow=True);

# Observing Constraints

In [0]:
from astroplan import AltitudeConstraint, AirmassConstraint, AtNightConstraint
from astroplan import observability_table

In [0]:
observing_range = [observe_start, observe_end]

In [0]:
constraints = [AirmassConstraint(2)]

In [0]:
observing_table = observability_table(constraints, my_location, targets, time_range=observing_range)

print(observing_table)

## Let us add another constraint

In [0]:
constraints.append(AltitudeConstraint(min=None, max=50*u.deg))

In [0]:
observing_table = observability_table(constraints, my_location, targets, time_range=observing_range)

print(observing_table)

## Additional Constraints

`from astroplan import CONSTRAINT`

* `AtNightConstraint()` - Constrain the Sun to be below horizon.
* `MoonIlluminationConstraint(min, max)` - Constrain the fractional illumination of the Moon.
* `MoonSeparationConstraint(min, max)` - Constrain the separation between the Moon and some targets.
* `SunSeparationConstraint(min, max)` - Constrain the separation between the Sun and some targets.

In [0]:
from astroplan import moon_illumination

In [0]:
moon_illumination(observe_start)

In [0]:
from astroplan import MoonSeparationConstraint

In [0]:
constraints.append(MoonSeparationConstraint(50*u.deg))

In [0]:
observing_table = observability_table(constraints, my_location, targets, time_range=observing_range)

print(observing_table)

In [0]:
fig,ax = plt.subplots(1,1)
fig.set_size_inches(10,5)

fig.tight_layout()

for i, my_object in enumerate(targets):

    if observing_table['ever observable'][i]:
        ax = plot_airmass(my_object, my_location, window_time_grid, max_airmass=2.0)

ax.vlines(observe_start.datetime, 1,3, color='r', linewidth=5)
ax.vlines(observe_end.datetime, 1,3, color='r', linewidth=5)

ax.legend(loc=0,shadow=True);

### Print out observing lengths for each object

In [0]:
for i, my_object in enumerate(targets):

    if observing_table['ever observable'][i]:
        name = observing_table['target name'][i]
        obj_frac = observing_table['fraction of time observable'][i]
        obj_time = obj_frac * observing_length
        output = "You can observe {0:s} for {1:.2f}".format(name, obj_time.to(u.h))
        print(output)